# BatchBacktesting

In [ ]:
# import library
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
import sys
import os
import httpx

In [ ]:
def make_api_request(api_endpoint, params):
    with httpx.Client() as client:
        # Make the GET request to the API
        response = client.get(api_endpoint, params=params)
        if response.status_code == 200:
            return response.json()
        print("Error: Failed to retrieve data from API")
        return None


In [ ]:
BASE_URL_FMP = "https://financialmodelingprep.com/api/v3"
BASE_URL_BINANCE = "https://fapi.binance.com/fapi/v1/"

FMP_API_KEY = "17c09553207c6d6e7bab10003e604aa8"
BINANCE_API_KEY = "17754325821w1b1e2khd34561hc54hh1"


def get_historical_price_full_crypto(symbol):
    api_endpoint = f"{BASE_URL_FMP}/historical-price-full/crypto/{symbol}"
    params = {"apikey": FMP_API_KEY}
    return make_api_request(api_endpoint, params)


def get_historical_price_full_stock(symbol):
    api_endpoint = f"{BASE_URL_FMP}/historical-price-full/{symbol}"
    params = {"apikey": FMP_API_KEY}

    return make_api_request(api_endpoint, params)

def get_SP500():
    api_endpoint = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    data = pd.read_html(api_endpoint)
    return list(data[0]['Symbol'])

def get_all_crypto():
    """
    All possible crypto on binance
    """

    api_endpoint = "https://api.binance.com/api/v3/ticker/price"
    data = make_api_request(api_endpoint, {})
    return [i['symbol'] for i in data]


def get_financial_statements_lists():
    api_endpoint = f"{BASE_URL_FMP}/financial-statement-symbol-lists"
    params = {"apikey": FMP_API_KEY}
    return make_api_request(api_endpoint, params)

In [ ]:
data_downloaded = get_historical_price_full_stock("AAPL")
data = data_downloaded["historical"]
data = pd.DataFrame(data)
data.columns = [x.title() for x in data.columns]  # uppercase first letter
data.drop(
    [
        "Adjclose",
        "Unadjustedvolume",
        "Change",
        "Changepercent",
        "Vwap",
        "Label",
        "Changeovertime",
    ],
    axis=1,
    inplace=True,
)
data["Date"] = pd.to_datetime(data["Date"])
data.set_index("Date", inplace=True)  # date needs to be set as index!
data = data.iloc[::-1]  # to reverse the order of the dataframe
data

In [ ]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import pandas_ta as taPanda


class Ema(Strategy):
    n1 = 20
    n2 = 80
    n3 = 150

    def init(self):
        close = self.data.Close
        self.ema20 = self.I(taPanda.ema, close.s, self.n1)
        self.ema80 = self.I(taPanda.ema, close.s, self.n2)
        self.ema150 = self.I(taPanda.ema, close.s, self.n3)

    def next(self):
        price = self.data.Close
        if crossover(self.ema20, self.ema80):
            self.position.close()
            self.buy(sl=0.90 * price, tp=1.25 * price)

        elif crossover(self.ema80, self.ema20):
            self.position.close()
            self.sell(sl=1.10 * price, tp=0.75 * price)


bt = Backtest(data, Ema, cash=100000, commission=0.002, exclusive_orders=True)
output = bt.run()

output = pd.DataFrame(pd.DataFrame(output).T)


In [ ]:
import concurrent.futures
import glob
import os
import warnings

warnings.filterwarnings("ignore")


def run_backtests(instruments, strategy=Ema, num_threads=4):
    """Run backtests for a list of instruments using a specified strategy."""

    outputs = []
    metric = "Equity Final [$]"
    # create the output directory if it doesn't exist
    output_dir = f"output/raw/{strategy.__name__}"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    def process_instrument(instrument):
        fileName = f"{output_dir}/{instrument}-*.csv"
        existingFiles = glob.glob(fileName)
        if existingFiles:
            print(f"{fileName} already exists. Skipping...")
            return None

        try: 
            data = get_historical_price_full_stock(instrument)
        except Exception:
            data = get_historical_price_full_crypto(instrument)

        try:
            data = data["historical"]
            data = pd.DataFrame(data)
            data.columns = [x.title() for x in data.columns]  # uppercase first letter
            data.drop(
                [
                    "Adjclose",
                    "Unadjustedvolume",
                    "Change",
                    "Changepercent",
                    "Vwap",
                    "Label",
                    "Changeovertime",
                ],
                axis=1,
                inplace=True,
            )
            data["Date"] = pd.to_datetime(data["Date"])
            data.set_index("Date", inplace=True)  # date needs to be set as index!
            data = data.iloc[::-1]  # to reverse the order of the dataframe

            # Create a backtest for the instrument using the specified strategy
            bt = Backtest(
                data,
                strategy=strategy,
                cash=100000,
                commission=0.002,
                exclusive_orders=True,
            )
            output = bt.run()
            output = pd.DataFrame(pd.DataFrame(output).T)

            # Add instrument name, strategy name, and parameters to output
            output["Instrument"] = instrument
            output["Strategy"] = strategy.__name__
            output.pop("_strategy")
            output["StrategyParameters"] = strategy.__dict__

            # Append output to list of outputs
            return output

        except Exception as e:
            print(f"Error processing {instrument}: {str(e)}")
            return None

    with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
        future_to_instrument = {
            executor.submit(process_instrument, instrument): instrument
            for instrument in instruments
        }
        for future in concurrent.futures.as_completed(future_to_instrument):
            instrument = future_to_instrument[future]
            output = future.result()
            if output is not None:
                outputs.append(pd.DataFrame(output))

                # Save output to file
                start = output["Start"].to_string().strip().split()[1]
                end = output["End"].to_string().strip().split()[1]
                fileNameOutput = f"{output_dir}/{instrument}-{start}-{end}.csv"
                output.to_csv(fileNameOutput)

                # Plot and save chart to file
                fileNameChart = f"{output_dir}/plot/{instrument}-{start}-{end}.html"
                bt.plot(filename=fileNameChart, open_browser=False)

        # Combine all the dataframes into one
        data_frame = pd.concat(outputs, ignore_index=True)

        # Save the data to a CSV file
        start = data_frame["Start"].to_string().strip().split()[1]
        end = data_frame["End"].to_string().strip().split()[1]
        fileNameOutput = f"output/{strategy.__name__}-{start}-{end}.csv"
        data_frame.to_csv(fileNameOutput)

    return outputs

In [ ]:
tickers = get_SP500()
run_backtests(tickers, strategy=Ema, num_threads=4)

In [ ]:
ticker = get_all_crypto()
run_backtests(ticker, strategy=Ema, num_threads=4)

In [ ]:
# run_backtests(get_financial_statements_lists(), Ema)

In [ ]:
class RSI(Strategy):
    n1 = 14
    n2 = 30
    n3 = 70

    def init(self):
        close = self.data.Close
        self.rsi = self.I(taPanda.rsi, close.s, self.n1)
        self.rsi30 = self.I(taPanda.rsi, close.s, self.n2)
        self.rsi70 = self.I(taPanda.rsi, close.s, self.n3)

    def next(self):
        price = self.data.Close
        if crossover(self.rsi, self.rsi30):
            self.position.close()
            self.buy(sl=0.90 * price, tp=1.25 * price)

        elif crossover(self.rsi70, self.rsi):
            self.position.close()
            self.sell(sl=1.10 * price, tp=0.75 * price)

    def optimize_parameters(self):
        self.n1 = self.I(taPanda.rsi, self.data.Close.s, self.n1)
        self.n2 = self.I(taPanda.rsi, self.data.Close.s, self.n2)
        self.n3 = self.I(taPanda.rsi, self.data.Close.s, self.n3)

    def optimize_run(self):
        self.optimize_parameters()
        self.run()


In [ ]:
tickers = get_SP500()
run_backtests(tickers, strategy=RSI, num_threads=4)

In [ ]:
ticker = get_all_crypto()
run_backtests(ticker, strategy=RSI, num_threads=4)

In [ ]:
from src.strategies import *

strategies = STRATEGIES

def run_backtests_strategies(instruments, strategies):
    """
    Run backtests for a list of instruments using a specified strategy.

    Args:
        instruments (list): List of instruments to run backtests for
        strategies (list): List of strategies to run backtests for

    Returns:
        List of outputs from run_backtests()

    """

    # find strategies in the STRATEGIES
    strategies = [x for x in STRATEGIES if x.__name__ in strategies]
    outputs = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for strategy in strategies:
            future = executor.submit(run_backtests, instruments, strategy, 4)
            futures.append(future)

        for future in concurrent.futures.as_completed(futures):
            outputs.extend(future.result())

    return outputs

In [ ]:
tickers = get_SP500()

run_backtests_strategies(tickers, ["LinearRegression"])

In [ ]:
tickers = get_all_crypto()

run_backtests_strategies(tickers, ["LinearRegression"])

In [ ]:
class LinearRegression(Strategy):
    n1 = 20
    n2 = 2

    def init(self):
        close = self.data.Close
        self.lreg = self.I(taPanda.linreg, close.s, self.n1, self.n2)

    def next(self):
        price = self.data.Close
        if crossover(price, self.linreg):
            self.position.close()
            self.buy(sl=0.90 * price, tp=1.25 * price)

        elif crossover(self.linreg, price):
            self.position.close()
            self.sell(sl=1.10 * price, tp=0.75 * price)

In [ ]:
tickers = get_SP500()
run_backtests(tickers, strategy=LinearRegression, num_threads=4)

In [ ]:
ticker = get_all_crypto()
run_backtests(ticker, strategy=LinearRegression, num_threads=4)